In [1]:
import sys
import os

src_path = os.path.join(os.path.dirname(os.getcwd()), 'src')
if src_path not in sys.path:
    sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

import kg, utils
from dotenv import load_dotenv
import pickle

In [2]:
kggen = kg.KnowledgeGraphGenerator(model = "openai/gpt-4o", api_key=os.getenv("OPENAI_API_KEY"))

# Manual Process

In [3]:
#file_name = "BenGurionFirstPassageWiki"
#file_name = "JerusalemWikiShort"
#file_name = "Ramban19_2"
#file_name = "IbnShuib19_2"
#file_name = "MeiratEinayim19_2"
#file_name = "KeterShemTov19_2"
#file_name = "Eskorial19_2"
#file_name = "Vatikan214_19_2"
#file_name = "London487_19_2"
file_name = "zfad"
chunk_size = 300


In [4]:
#path = "../../Ramban/"
path = "/Users/hadarmiller/Downloads/Eliezer/"
with open(f"{path}txt/{file_name}.txt", "r", encoding="utf-8") as file:
    text = file.read()
    text = text.strip().replace("\n", " ")

#text

In [5]:
#text.split(".")

In [8]:
kggen.extractConcepts(text=text, chunk_size=chunk_size, verbose=True)


In [7]:
#kggen.graph.concepts

In [8]:
kggen.extractRelations(verbose = True)

Dropped non-English predicate: חיבר
Number of RDF relations: 125


In [16]:
#kggen.graph.relations

In [6]:
kggen.groupConcepts(chunk_size= 160, threshold= 160, max_iterations= 2, verbose=True)

[Round 1] Grouping 159 concepts in parallel batches...
[Round 1] Reduced to 141 concepts.
[Round 2] Grouping 141 concepts in parallel batches...
[Round 2] Reduced to 140 concepts.
[Round 3] Grouping 140 concepts in parallel batches...
[Round 3] Reduced to 139 concepts.
Reached maximum iterations (3). Stopping early.
Number of concepts after grouping: 139


In [11]:
#kggen.graph.concepts

In [12]:
#recommendation = kggen.ontologyRecommendation()
#print(recommendation)

In [8]:
kggen.relations2ontology(["SKOS"]) # "DUBLIN_CORE", "SKOS", "CDOCCRM"

In [9]:
kggen.relations2ontology(["DUBLIN_CORE"])

In [10]:
kggen.relations2ontology(["CDOCCRM"])

In [11]:
kggen.relations2ontology(["LLM"])

In [12]:
kggen.relations2ontology(["CDOCCRM", "SKOS", "DUBLIN_CORE"])

In [12]:
#kggen.graph.ontologies

In [13]:
kggen.dump_graph(f"{path}json/{file_name}_c{chunk_size}.json")
#kggen.dump_graph(f"/Users/hadarmiller/Downloads/Ramban/json/JerOnePassage.json")

In [14]:
kggen.init_graph(path=f"{path}json/{file_name}_c{chunk_size}.json")

In [23]:
ontology = "MIX"
kggen.graph2Ontology(ontology)

In [24]:
viz = kggen.visualize(f"{path}vis/{file_name}_c{chunk_size}_ontology_{ontology}.html")
#viz = kggen.visualize()

In [15]:
viz = kggen.visualize()
viz.show("results.html")

results.html


In [21]:
#kggen.graph.entities

In [33]:
#kggen.graph.concepts

In [6]:
term = "כרובים על הפרכת"
term = "קדש הקדשים"
tmp_rel = [x[0] for x in kggen.graph.relations] + [x[2] for x in kggen.graph.relations]
cmp_conc = [x.prefLabel_he for x in kggen.graph.concepts]
tmp_con_alt = []
for c in kggen.graph.concepts:
    tmp_con_alt += c.altLabels_he
print(f"The term in Entities: {term in kggen.graph.entities}\nThe term in Relations: {term in tmp_rel}\nThe term in Concepts: {term in cmp_conc}")
print(f"The term in Concepts ALTs: {term in tmp_con_alt}")

The term in Entities: True
The term in Relations: True
The term in Concepts: True
The term in Concepts ALTs: False


In [11]:
def mergeNodesEdges(prefLabel: str, altLabels: list[str]):

    relations = list(kggen.graph.relations)
    # Filter out relations where subject or object is in altLabels
    # and replace them with the prefLabel.
    # This ensures that all relations involving altLabels are replaced with the prefLabel.
    # This is done to unify the entities under a single concept.
    new_relations = [x for x in relations if x[0] not in  altLabels and x[2] not in altLabels]

    for al in altLabels:
        new_relations.extend([(prefLabel,r,s) for o,r,s in relations if o==al])
        new_relations.extend([(o,r,prefLabel) for o,r,s in relations if s==al])
    

    new_nodes = [x for x in kggen.graph.entities if x not in altLabels]

    return new_relations

In [12]:
tmp = kggen.graph.concepts[2]

rel_tmp = []
for r in kggen.graph.relations:
    rel_tmp.append(r[0])
    rel_tmp.append(r[2])
print(term in rel_tmp)

ttt = mergeNodesEdges(tmp.prefLabel_he, tmp.altLabels_he)
tmp_con_alt2 = []
for c in ttt:
    tmp_con_alt2.append(c[0])
    tmp_con_alt2.append(c[2])
print(term in tmp_con_alt2)


True
True


In [3]:
#kggen.graph.concepts

# Full Process

In [3]:
#file_name = "JerusalemWikiShort"
#file_name = "BenGurionFirstPassageWiki"
#file_name = "Ramban19_2"
file_name = "IbnShuib19_2"
#file_name = "MeiratEinayim19_2" #ERROR
#file_name = "KeterShemTov19_2"
#file_name = "Eskorial19_2"
#file_name = "Vatikan214_19_2"
#file_name = "London487_19_2"

chunk_size = 300
text_path = "../../Ramban/txt/"
text_file_name = file_name + ".txt"
ontologies = ["SKOS", "DUBLIN_CORE", "CDOCCRM", "LLM"]
viz_path = "../../Ramban/vis/"

verbose = True

In [4]:
kggen.generateKG(text_path, text_file_name, ontologies, viz_path, chunk_size, verbose)

Number of subgraphs: 9
Number of Entities: 62
Entities list: כרוביו, מקדש, תבנית, פניהם איש אל אחיו, העדות, משכן, אלהי ישראל, עשר מנורות, ארון ברית ה', עצי הלבנון, כרובים, פרק המוכר פירות, מרכבה, תורה שבכתב, הרב ז"ל, המעלה, הבנין, ייחוד, הארון, הלוחות, נושאי הכבוד, נהר כבר, ארון הברית, הכרובים, חיה, חמש אמות, עשר כיורים, תפארת, שפע היורד, מורי נר"ו, חמש האחד, עליונים, תבנית לכרובים העליונים, חמש משמאל, בנין, זהב, כבוד, חמש מימין, עין בעין, חמש כנגד חמש, הכפורת, העניינים הרוחנים, היחוד, מכונות, שם הגדול, יושב הכרובים, רצון השלם, אברהם, מרכבת יחזקאל, מורי י"א, כנף האחד, דו פרצופים, שפע, הצואה, צדק ושלום נשקו, מקשה, כסא כבוד, החיה, תורה שבע"פ, גבוהים, יחזקאל, שלמה המלך
Number of subgraphs: 9
Number of RDF relations: 47
validateRelations-->Found invalid relations after merging concepts: {'מקדש', 'תבנית', 'עשר מנורות', 'העדות', 'פניהם איש אל אחיו', 'אלהי ישראל', 'עצי הלבנון', 'כרובים', 'מרכבה', 'תורה שבכתב', 'הרב ז"ל', 'הבנין', 'ייחוד', 'הלוחות', 'נושאי הכבוד', 'נהר כבר', 'ארון הברית', 'עשר

In [5]:
kggen.dump_graph(f"../../Ramban/json/{file_name}_c{chunk_size}.json")